# Part 1, Topic 1: Introduction to Clock Glitching (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Microcontrollers and FPGAs have a number of operating conditions that must be met in order for the device to work properly. Outside of these conditions, devices will begin to malfunction, with more extreme violations causing the device to stop entirely or even become damaged. By going outside these operating conditions for very small amounts of time, we can cause a varitey of temporary malfunctions*

*In this lab, we'll explore clock glitching, which inserts short glitches into a device's clock. This will be used to get a target that's summing numbers in a loop to arrive at the wrong result.*

**LEARNING OUTCOMES:**

* Understand effects of clock glitching
* Exploring ChipWhisperer's glitch module
* Using clock glitching to disrupt a target's algorithm

## Clock Glitching Theory

Digital hardware devices almost always expect some form of reliable clock. We can manipulate the clock being presented to the device to cause unintended behaviour. We'll be concentrating on microcontrollers here, however other digital devices (e.g. hardware encryption accelerators) can also have faults injected using this technique.

Consider a microcontroller first. The following figure is an excerpt from the Atmel AVR ATMega328P datasheet:

![A2_1](img/Mcu-unglitched.png)

Rather than loading each instruction from FLASH and performing the entire execution, the system has a pipeline to speed up the execution process. This means that an instruction is being decoded while the next one is being retrieved, as the following diagram shows:

![A2_2](img/Clock-normal.png)

But if we modify the clock, we could have a situation where the system doesn't have enough time to actually perform an instruction. Consider the following, where Execute #1 is effectively skipped. Before the system has time to actually execute it another clock edge comes, causing the microcontroller to start execution of the next instruction:

![A2_3](img/Clock-glitched.png)

This causes the microcontroller to skip an instruction. Such attacks can be immensely powerful in practice. Consider for example the following code from `linux-util-2.24`:

```C
/*
 *   auth.c -- PAM authorization code, common between chsh and chfn
 *   (c) 2012 by Cody Maloney <cmaloney@theoreticalchaos.com>
 *
 *   this program is free software.  you can redistribute it and
 *   modify it under the terms of the gnu general public license.
 *   there is no warranty.
 *
 */

#include "auth.h"
#include "pamfail.h"

int auth_pam(const char *service_name, uid_t uid, const char *username)
{
    if (uid != 0) {
        pam_handle_t *pamh = NULL;
        struct pam_conv conv = { misc_conv, NULL };
        int retcode;

        retcode = pam_start(service_name, username, &conv, &pamh);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_authenticate(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_acct_mgmt(pamh, 0);
        if (retcode == PAM_NEW_AUTHTOK_REQD)
            retcode =
                pam_chauthtok(pamh, PAM_CHANGE_EXPIRED_AUTHTOK);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_setcred(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        pam_end(pamh, 0);
        /* no need to establish a session; this isn't a
         * session-oriented activity...  */
    }
    return TRUE;
}
```

This is the login code for the Linux OS. Note that if we could skip the check of `if (uid != 0)` and simply branch to the end, we could avoid having to enter a password. This is the power of glitch attacks - not that we are breaking encryption, but simply bypassing the entire authentication module! 

### Glitch Hardware

The ChipWhisperer Glitch system uses the same synchronous methodology as its Side Channel Analysis (SCA) capture. A system clock (which can come from either the ChipWhisperer or the Device Under Test (DUT)) is used to generate the glitches. These glitches are then inserted back into the clock, although it's possible to use the glitches alone for other purposes (i.e. for voltage glitching, EM glitching).

The generation of glitches is done with two variable phase shift modules, configured as follows:

![A2_4](img/Glitchgen-phaseshift.png)

In CW-Husky there is one important difference: the phase shift 1 output is not inverted before it is ANDed with the phase shift 2 output.

The enable line is used to determine when glitches are inserted. Glitches can be inserted continuously (useful for development) or triggered by some event. The following figure shows how the glitch can be muxd to output to the Device Under Test (DUT).

![A2_5](img/Glitchgen-mux.png)

### Hardware Support: CW-Lite/Pro

The phase shift blocks use the Digital Clock Manager (DCM) blocks within the FPGA. These blocks have limited support for run-time configuration of parameters such as phase delay and frequency generation, and for maximum performance the configuration must be fixed at design time. The Xilinx-provided run-time adjustment can shift the phase only by about +/- 5nS in 30pS increments (exact values vary with operating conditions).

For most operating conditions this is insufficient - if attacking a target at 7.37MHz the clock cycle would have a period of 136nS. In order to provide a larger adjustment range, an advanced FPGA feature called Partial Reconfiguration (PR) is used. The PR system requires special partial bitstreams which contain modifications to the FPGA bitstream. These are stored as two files inside a "firmware" zip which contains both the FPGA bitstream along with a file called `glitchwidth.p` and a file called `glitchoffset.p`. If a lone bitstream is being loaded into the FPGA (i.e. not from the zip-file), the partial reconfiguration system is disabled, as loading incorrect partial reconfiguration files could damage the FPGA. This damage is mostly theoretical, more likely the FPGA will fail to function correctly.

If in the course of following this tutorial you find the FPGA appears to stop responding (i.e. certain features no longer work correctly), it could be the partial reconfiguration data is incorrect.

We'll look at how to interface with these features later in the tutorial.

### Hardware Support: CW-Husky

The clock-generation logic in Husky's 7-series FPGA is considerably different than the 6-series FPGAs used in CW-Lite/Pro. The DCM is gone and replaced by the much more powerful (and power hungry...) Mixed Mode Clock Manager (MMCM). In particular for our glitching application, MMCMs allow fine phase shift adjustments over an unlimited range, in steps as small as 15ps. And all this without having to dynamically reconfigure the FPGA bitfile! For this reason, the format for specifying the glitch offset and width is different from what it was for CW-Lite/Pro. Instead of specifiying a percentage of the source clock period, you now specify the actual number of phase shift steps. The duration of one phase shift step is 1/56 of the MMCM VCO clock period, which can itself be configured to be anyhwere in the range from 600 MHz to 1200 MHz (via `scope.clock.pll.update_fpga_vco()`).

While the MMCM is more powerful than the DCM with respect to its features, it also requires a lot more power. For this reason, the glitch generation circuitry is disabled by default and must be explicitly turned on. Fear not, Husky also uses Xilinx's XADC module to continuously monitor its temperature, and all MMCMs are automatically turned off at when the temperature starts getting too high, well below dangerous levels are reached (run `scope.XADC` to see all its statistics and settings).


In [34]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [35]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 13896489                  to 38384450                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 73644767                 
scope.clock.adc_rate                     changed from 0.0                       to 73644767.0        

In [36]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f3/stm32f3_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWLITEARM/stm32f3_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f3 -I.././hal/stm32f3/CMSIS -I.././hal/stm32f3/CMSIS/core -I.././hal/stm32f3/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f3/stm32f3_startup.S -o objdir-

In [37]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=='SS_VER_2_1':
    target.reset_comms()

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5835 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5835 bytes


We'll probably crash the target a few times while we're trying some glitching. Create a function to reset the target:

In [38]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

## Communication

For this lab, we'll be introducing a new method: `target.simpleserial_read_witherrors()`. We're expecting a simpleserial response back; however, glitch will often cause the target to crash and return an invalid string. This method will handle all that for us. It'll also tell us whether the response was valid and what the error return code was. Use as follows:

In [39]:
#Do glitch loop
target.simpleserial_write("g", bytearray([]))

val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

#print(bytearray(val['full_response'].encode('latin-1')))
print(val)

{'valid': True, 'payload': CWbytearray(b'c4 09 00 00'), 'full_response': CWbytearray(b'00 72 04 c4 09 00 00 15 00'), 'rv': bytearray(b'\x00')}


## Target Firmware

For this lab, our goal is to get the following code to preduce an incorrect result:

```C
uint8_t glitch_loop(uint8_t* in)
{
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    cnt = 0;
    trigger_high();
    for(i=0; i<50; i++){
        for(j=0; j<50; j++){
            cnt++;
        }
    }
    trigger_low();
    simpleserial_put('r', 4, (uint8_t*)&cnt);
    return (cnt != 2500);
}
```

As you can see, we've got a simple loop. This is a really good place to start glitching for 2 reasons:

1. We've got a really long portion of time with a lot of instructions to glitch. In contrast, with the Linux example we're be trying to target a single instruction.

1. For some glitching scenarios, we're looking for a pretty specific glitch effect. In the Linux example, we might be banking on the glitch causing the target to skip an instruction instead of corrupting the comparison since that's a lot more likely to get us where we want in the code path. For this simple loop calculation, pretty much any malfunction will show up in the result.

## Glitch Module

All the settings/methods for the glitch module can be accessed under `scope.glitch`. As usual, documentation for the settings and methods can be accessed on [ReadtheDocs](https://chipwhisperer.readthedocs.io/en/latest/api.html) or with the python `help` command:

In [25]:
help(scope.glitch)

Help on GlitchSettings in module chipwhisperer.capture.scopes.cwhardware.ChipWhispererGlitch object:

class GlitchSettings(chipwhisperer.common.utils.util.DisableNewAttr)
 |  GlitchSettings(cwglitch)
 |  
 |  Method resolution order:
 |      GlitchSettings
 |      chipwhisperer.common.utils.util.DisableNewAttr
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cwglitch)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  manualTrigger(self)
 |  
 |  manual_trigger(self)
 |      Manually trigger the glitch output.
 |      
 |      This trigger is most useful in Manual trigger mode, where this is the
 |      only way to cause a glitch.
 |      
 |      Note that for ChipWhisperer-Husky, this method will only cause a glitch
 |      in manual mode, while on the Lite/Pro, this method will always insert a glitch.
 |  
 |  readStatus(self)

#### We'll first go over settings that differ between the CW Husky and the CW Lite/Pro:
* clk_src

> The clock signal that the glitch DCM is using as input. Can be set to "target" or "clkgen" In this case, we'll be providing the clock to the target, so we'll want this set to "clkgen".

> On CW Husky, a separate PLL is used to clock the glitch module instead of the clkgen module. The equivalent setting here for "clkgen" is "pll"
* offset

> Where in the output clock to place the glitch. Can be in the range `[-48.8, 48.8]`. Often, we'll want to try many offsets when trying to glitch a target.

> On CW Husky, the range will depend on frequency of the PLL used to drive the glitch module (settable which can be configured to be anyhwere in the range from 600 MHz to 1200 MHz via `scope.clock.pll.update_fpga_vco()`), but, when the glitch module is active, the range will be `[0, scope.glitch.phase_shift_steps]`.
* width

> How wide to make the glitch. Can be in the range `[-50, 50]`, **though there is no reason to use widths < 0**. Wider glitches more easily cause glitches, but are also more likely to crash the target, meaning we'll often want to try a range of widths when attacking a target.

> Like offset, the range will be `[0, scope.glitch.phase_shift_steps]`.

#### These settings, on the other hand, are the same between the Husky and the Lite/Pro:

* output

> The output produced by the glitch module. For clock glitching, clock_xor is often the most useful option, as this inverts the clock during the glitch.
* ext_offset

> The number of clock cycles after the trigger to put the glitch.
* repeat

> The number of clock cycles to repeat the glitch for. Higher values increase the number of instructions that can be glitched, but often increase the risk of crashing the target.

* trigger_src

> How to trigger the glitch. For this tutorial, we want to automatically trigger the glitch from the trigger pin only after arming the ChipWhipserer, so we'll use `ext_single`

In addition, we'll need to tell ChipWhipserer to use the glitch module's output as a clock source for the target by setting `scope.io.hs2 = "glitch"`. We'll also setup a large `repeat` to make glitching easier.

## CW Glitch Controller

To make creating a glitch loop easier, ChipWhisperer includes a glitch controller. We'll start of by initializing with with different potential results of the attack. You define these to be whatever you want, but often three groups are sufficient:

1. `"success"`, where our glitch had the desired effect
1. `"reset"`, where our glitch had an undesirable effect. Often, this effect is crashing or resetting the target, which is why we're calling it `"reset"`
1. `"normal"`, where you glitch didn't have a noticable effect.

We also need to tell it what glitch parameters we want to scan through, in this case width and offset:

In [40]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "tries"])

One of the niceties of the glitch controller is that it can display our current settings. This will update in real time as we use the glitch controller!

In [63]:
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=48.0, readout…

FloatSlider(value=-48.0, continuous_update=False, description='offset setting:', disabled=True, max=48.0, min=…

FloatSlider(value=1.0, continuous_update=False, description='tries setting:', disabled=True, max=1.0, min=1.0,…

We can also make a settings plot that can also update in realtime as well:

In [42]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [width,offset]
      .Points.II :Points   [width,offset]

Here `plotdots` is a dictionary that specifies how you want to plot each group. In this case, we're plotting `"success"` as a green `+` (`"+g"`), `"reset"` as a red `x` (`"xr"`), and we won't be plotting glitch attempts where nothing abnormal happens (`None`)

This plot will auto update its bounds as points are added. If you want to specify the axis bounds, you can do so as follows:

```python
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_bound=(-48, 48), y_bound=(-48, 48))
```

You can also select which parameters you want to use for x and y, either by index, or by its name:

```python
# will flip width and offset axes
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index=1, y_index=0)
# or
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="offset", y_index="width")

```

You can set ranges for each glitch setting:

In [43]:
gc.set_range("width", -5, 5)
gc.set_range("offset", -5, 5)

Each setting moves from min to max based on the global step:

In [44]:
gc.set_global_step([5.0, 2.5])

We can print out all the glitch settings to see how this looks:

In [45]:
for glitch_setting in gc.glitch_values():
    print("offset: {:4.1f}; width: {:4.1f}".format(glitch_setting[1], glitch_setting[0]))

offset: -5.0; width: -5.0
offset: -5.0; width: -5.0
offset: -5.0; width: -5.0
offset:  0.0; width: -5.0
offset:  0.0; width: -5.0
offset:  0.0; width: -5.0
offset:  5.0; width: -5.0
offset:  5.0; width: -5.0
offset:  5.0; width: -5.0
offset: -5.0; width:  0.0
offset: -5.0; width:  0.0
offset: -5.0; width:  0.0
offset:  0.0; width:  0.0
offset:  0.0; width:  0.0
offset:  0.0; width:  0.0
offset:  5.0; width:  0.0
offset:  5.0; width:  0.0
offset:  5.0; width:  0.0
offset: -5.0; width:  5.0
offset: -5.0; width:  5.0
offset: -5.0; width:  5.0
offset:  0.0; width:  5.0
offset:  0.0; width:  5.0
offset:  0.0; width:  5.0
offset:  5.0; width:  5.0
offset:  5.0; width:  5.0
offset:  5.0; width:  5.0
offset: -5.0; width: -5.0
offset: -5.0; width: -5.0
offset: -5.0; width: -5.0
offset: -5.0; width: -5.0
offset: -5.0; width: -5.0
offset: -2.5; width: -5.0
offset: -2.5; width: -5.0
offset: -2.5; width: -5.0
offset: -2.5; width: -5.0
offset: -2.5; width: -5.0
offset:  0.0; width: -5.0
offset:  0.0

You can tell the glitch controller when you've reached a particular result state like so:

In [46]:
#gc.add("reset", (scope.glitch.width, scope.glitch.offset)) or simply gc.add("reset")
gc.add("reset")
#gc.add("success", (scope.glitch.width, scope.glitch.offset)) or simply gc.add("success")
gc.add("success")

As of ChipWhisperer 5.7, you can skip the glitch width and glitch offset parameters. In this case, the glitch controller will use its internal values for the coordinates. Note that due to rounding, this will usually be a bit different from the actual hardware value on the Lite/Pro; however, the values will still correspond to the correct settings on your ChipWhisperer.

For CW-Husky, we must first explicitly turn on the glitch circuitry (it is off by default for power savings):

In [47]:
if scope._is_husky:
    scope.glitch.enabled = True

We'll start off with the following settings. It's usually best to use "clock_xor" with clock glitching, which will insert a glitch if the clock is high or the clock is low.

In [48]:
#Basic setup
# set glitch clock
if scope._is_husky:
    scope.glitch.clk_src = "pll"
else:
    scope.glitch.clk_src = "clkgen" 

scope.glitch.output = "clock_xor" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called

scope.io.hs2 = "glitch"  # output glitch_out on the clock line
print(scope.glitch)

clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 1
output      = clock_xor



These settings are often a good starting point for all clock glitching, so, new with ChipWhisperer 5.7, we've got a method that sets all of this up for you:

In [1]:
#scope.cglitch_setup()

You should have all you need to construct your glitch loop. We'll get you started, but the rest is up to you! Also, some stuff to keep in mind:

* You'll need to detect crashes, successful glitches, and normal returns from the target. Don't be afraid to experiment with the loop: you can always restart it and rerun the code.
* You can cover a larger set of glitch settings by starting with large glitch controller steps to get idea where some interesting locations are, then repeating the glitch loop with small steps in interesting areas. Where there's one successful glitch, there's probably more!
* You can speed up your glitch campaign substantially by only plotting crashes and successes, since they're typically much rarer than normal behaviour in the target
* On CW-Husky, glitch offset and width are specified in number of phase shift steps, whereas on CW-Lite/Pro, they are specified in percentage of clock period. The code provided below sets appropriate starting ranges for each case. Run `help(scope.glitch)` to understand this better.

In [69]:
import chipwhisperer.common.results.glitch as glitch
from chipwhisperer import logging
from tqdm.notebook import trange
import struct

cw.glitch_logger.setLevel(logging.ERROR)
scope.glitch.ext_offset = 8

# width and offset numbers have a very different meaning for Husky vs Lite/Pro;
# see help(scope.glitch) for details
num_tries = 5
gc.set_range("tries", 1, num_tries)
if scope._is_husky:
    gc.set_range("width", 0, scope.glitch.phase_shift_steps)
    gc.set_range("offset", 0, scope.glitch.phase_shift_steps)
    gc.set_global_step([400, 200, 100])
    scope.adc.lo_gain_errors_disabled = True
    scope.adc.clip_errors_disabled = True
else:
    gc.set_range("width", 0, 48)
    gc.set_range("offset", -48, 48)
    gc.set_global_step([8, 4, 2, 1])

scope.glitch.repeat = 10
gc.set_step("tries", 1)
scope.adc.timeout = 0.1

# with open('glitch_values.txt', 'w') as f:
#     for glitch_setting in gc.glitch_values():
#         f.write(("offset: {:4.1f}; width: {:4.1f}".format(glitch_setting[1], glitch_setting[0])))

In [70]:
reboot_flush()

for glitch_setting in gc.glitch_values():
    
    # optional: you can speed up the loop by checking if the trigger never went low
    #           (the target never called trigger_low();) via scope.adc.state
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]

    if scope.adc.state:
        print("Trigger still high!")
        gc.add("reset")
        reboot_flush()

    scope.arm()
    
    target.simpleserial_write("g", bytearray([]))
    
    ret = scope.capture()  # True if capture timed out, false if it didn't.
    
    if ret: #here the trigger never went high - sometimes the target is still crashed from a previous glitch
        print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
        
        if not val['valid']: # change this to detect an invalid response
            gc.add("reset")
        else:
            # gcnt is the loop counter
            gcnt = struct.unpack("<I", val['payload'])[0]
            
            if gcnt == 2500: #normal response
                gc.add("normal")
            else: #glitch!!!
                gc.add("success")
                print(gcnt)
                print("offset: {:4.1f}; width: {:4.1f}".format(scope.glitch.offset, scope.glitch.width))
                print("🐙", end="")

(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff data
(ChipWhisperer Target ERROR|File SimpleSerial2.py:185) CWbytearray(b'00 00 00 00 00 00 00 00 00')
(ChipWhisperer Target WARNING|File SimpleSerial2.py:377) Unexpected start to command 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 0, 4


Trigger still high!


(ChipWhisperer Target ERROR|File SimpleSerial2.py:189) CWbytearray(b'00 72 01 00 99 00 03 65 01'), 5, 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 1, 4
(ChipWhisperer Target ERROR|File SimpleSerial2.py:189) CWbytearray(b'00 72 01 00 99 00 03 65 01'), 5, 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 1, 4
(ChipWhisperer Target ERROR|File SimpleSerial2.py:189) CWbytearray(b'00 72 01 00 99 00 03 65 01'), 5, 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 1, 4
(ChipWhisperer Target ERROR|File SimpleSerial2.py:189) CWbytearray(b'00 72 01 00 99 00 03 65 01'), 5, 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 1, 4
(ChipWhisperer Target ERROR|File SimpleSerial2.py:189) CWbytearray(b'00 72 01 00 99 00 03 65 01'), 5, 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 1, 4
(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff 

Trigger still high!


(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff data
(ChipWhisperer Target ERROR|File SimpleSerial2.py:185) CWbytearray(b'00 00 00 00 00 00 00 00 00')
(ChipWhisperer Target WARNING|File SimpleSerial2.py:377) Unexpected start to command 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 0, 4


...

Trigger still high!


(ChipWhisperer Target ERROR|File SimpleSerial2.py:288) Device reported error 0x10
(ChipWhisperer Target ERROR|File SimpleSerial2.py:290) CWbytearray(b'00 65 01 10 42 00')
(ChipWhisperer Target ERROR|File SimpleSerial2.py:288) Device reported error 0x10


2498
offset: -2.0; width: 46.9
🐙

(ChipWhisperer Target ERROR|File SimpleSerial2.py:290) CWbytearray(b'00 65 01 10 42 00')


2501
offset: -2.0; width: 46.9
🐙

### Results

In addition to plotting, the glitch controller also has the capability to return results as a list that groups paramters and results. These results give both the number of each result, as well as the rate of each result:

In [71]:
gc.calc()

[((0, -48, 1),
  {'total': 4,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 4,
   'normal_rate': 1.0}),
 ((0, -48, 2),
  {'total': 4,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 4,
   'normal_rate': 1.0}),
 ((0, -48, 3),
  {'total': 4,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 4,
   'normal_rate': 1.0}),
 ((0, -48, 4),
  {'total': 4,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 4,
   'normal_rate': 1.0}),
 ((0, -48, 5),
  {'total': 4,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 4,
   'normal_rate': 1.0}),
 ((0, -40, 1),
  {'total': 4,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 4,
   'normal_rate': 1.0}),
 ((0, -40, 2),
  {'total': 4,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0

You can also get results back with some parameters ignored. Results from parameters that now match will be grouped. This is particularly useful with something like the `"tries"` parameter, as you don't typically care whether a glitch was successful on your first, second, or third attempt:

In [72]:
results = gc.calc(ignore_params="tries")
results

[((0, -48),
  {'total': 20,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 20,
   'normal_rate': 1.0}),
 ((0, -40),
  {'total': 20,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 20,
   'normal_rate': 1.0}),
 ((0, -32),
  {'total': 20,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 20,
   'normal_rate': 1.0}),
 ((0, -24),
  {'total': 20,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 20,
   'normal_rate': 1.0}),
 ((0, -16),
  {'total': 20,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 20,
   'normal_rate': 1.0}),
 ((0, -8),
  {'total': 20,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 20,
   'normal_rate': 1.0}),
 ((0, 0),
  {'total': 20,
   'success': 0,
   'success_rate': 0.0,
   'reset': 0,
   'reset_rate': 0.0,
   'norma

Finally, `calc()` can also sort by different results. A common use for this is to sort by success rate:

In [73]:
results = gc.calc(ignore_params="tries", sort="success_rate")
results

[((4, -13),
  {'total': 5,
   'success': 4,
   'success_rate': 0.8,
   'reset': 1,
   'reset_rate': 0.2,
   'normal': 0,
   'normal_rate': 0.0}),
 ((12, -6),
  {'total': 11,
   'success': 8,
   'success_rate': 0.7272727272727273,
   'reset': 2,
   'reset_rate': 0.18181818181818182,
   'normal': 1,
   'normal_rate': 0.09090909090909091}),
 ((10, 2),
  {'total': 11,
   'success': 8,
   'success_rate': 0.7272727272727273,
   'reset': 3,
   'reset_rate': 0.2727272727272727,
   'normal': 0,
   'normal_rate': 0.0}),
 ((8, -12),
  {'total': 15,
   'success': 10,
   'success_rate': 0.6666666666666666,
   'reset': 2,
   'reset_rate': 0.13333333333333333,
   'normal': 3,
   'normal_rate': 0.2}),
 ((13, -11),
  {'total': 5,
   'success': 3,
   'success_rate': 0.6,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 2,
   'normal_rate': 0.4}),
 ((11, 1),
  {'total': 5,
   'success': 3,
   'success_rate': 0.6,
   'reset': 2,
   'reset_rate': 0.4,
   'normal': 0,
   'normal_rate': 0.0}),
 ((7, -12),
 

### Plotting Glitch Results

We can replot our glitch map using the `plot_2d()` method. Settings are similar to `glitch_plot()`. If `plotdots` are not specified, the same ones as the `glitch_plot()` will be used.

In [74]:
gc.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None})
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

:Overlay
   .Points.I       :Points   [x,y]
   .Points.Success :Points   [width,offset]   (success_rate)
   .Points.Reset   :Points   [width,offset]   (reset_rate)

Make sure you write down those glitch settings, since we'll be using for the rest of the glitching labs! In fact, we'll be using a lot of the general code structure here for the rest of the labs, with the only big changes being:

### Repeat

This lab used a pretty large repeat value. Like the name suggests, this setting controls how many times the glitch is repeated (i.e. a repeat value of 5 will place glitches in 5 consecutive clock cycles). Consider that each glitch inserted has a chance to both cause a glitch or crash the device. This was pretty advantageous for this lab since we had a lot of different spots we wanted to place a glitch - using a high repeat value increased our chance for a crash, but also increased our chance for a successful glitch. For an attack where we're targeting a single instruction, we don't really increase our glitch chance at all, but still have the increased crash risk. Worse yet, a successful glitch in a wrong spot may also cause a crash! It is for that reason that it's often better to use a low repeat value when targeting a single instruction.

### Ext Offset

The ext offset setting controls a delay between the trigger firing and the glitch being inserted. Like repeat, it's based on whole clock cycles, meaning an ext offset of 10 will insert a glitch 10 cycles after the trigger fires. We didn't have to worry about this setting for this lab since the large repeat value was able to take us into the area we wanted. This won't be true for many applications, where you'll have to try glitches at a large variety of ext_offsets.

### Success, Reset, and Normal

These three result states are usually enough to describe most glitch results. What constitues a success, however, will change based on what firmware you're attacking. For example, if we were attacking the Linux authentication, we might base success on a check to see whether or not we're root.

In [76]:
scope.dis()
target.dis()

(ChipWhisperer Scope ERROR|File naeusbchip.py:108) Scope already disconnected!
